In [1]:

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pyfolio as pf
import warnings
warnings.filterwarnings("ignore")
import argparse
import collections

/home/lokendra/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/home/lokendra/.local/lib/python3.6/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:

class TestStrategy(bt.Strategy):
    
    params = dict(profit_percent=0.3, loss_percent = 0.2, efast = 10, eslow=30)
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.price = None
        self.comm = None
        emafast = bt.ind.EMA(period=self.p.efast)
        emaslow = bt.ind.EMA(period=self.p.eslow)
        self.crossover = bt.ind.CrossOver(emafast,emaslow)
        
    def notify_order(self, order):
        date = self.data.datetime.datetime().date()
        
        if order.status in [order.Submitted, order.Accepted]:
            return
            
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.price = order.executed.price
                self.comm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Margin, order.Rejected]:
            self.log('Order Margin/Rejected')
        
        elif order.status in [order.Canceled]:
            self.log('Order Canceled')
            #Closing Bracket orders comes under this condition

        self.order = None
def next(self):
        #self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            #Check the order while placing the amounts for different types, limit order and to do functions
            #like find a breakout and confirm the indicator. Many a times generates errors
            if self.crossover>0:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                entry = self.dataclose
                self.order = self.buy_bracket(limitprice=entry*(1+self.p.profit_percent),
                                              stopprice=entry*(1-self.p.loss_percent),
                                              exectype=bt.Order.Market)

        #Uncomment this and find that it is buying and selling way too frequently.
        #The behavior can be observed from the plot but no explanation
        else:
            if self.crossover<0:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                entry = self.dataclose
                self.order = self.sell_bracket(limitprice=entry*(1-self.p.profit_percent),
                                              stopprice=entry*(1+self.p.loss_percent),
                                              exectype=bt.Order.Market)

In [3]:
def printTradeAnalysis(analyzer):
    
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("\n\nTrade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [4]:
MAINSIGNALS = collections.OrderedDict(
    (('longshort', bt.SIGNAL_LONGSHORT),
     ('longonly', bt.SIGNAL_LONG),
     ('shortonly', bt.SIGNAL_SHORT),)
)


EXITSIGNALS = {
    'longexit': bt.SIGNAL_LONGEXIT,
    'shortexit': bt.SIGNAL_LONGEXIT,
}


class SMACloseSignal(bt.Indicator):
    lines = ('signal',)
    params = (('period', 30),)

    def __init__(self):
        self.lines.signal = self.data - bt.indicators.SMA(period=self.p.period)


class SMAExitSignal(bt.Indicator):
    lines = ('signal',)
    params = (('p1', 5), ('p2', 30),)

    def __init__(self):
        sma1 = bt.indicators.SMA(period=self.p.p1)
        sma2 = bt.indicators.SMA(period=self.p.p2)
        self.lines.signal = sma1 - sma2


In [8]:
def parse_args(pargs=None):

    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Sample for Signal concepts')

    parser.add_argument('--data', required=False,
                        default='/home/lokendra/Downloads/orcl-1995-2014.txt',
                        help='Specific data to be read in')

    parser.add_argument('--fromdate', required=False, default=None,
                        help='Starting date in YYYY-MM-DD format')

    parser.add_argument('--todate', required=False, default=None,
                        help='Ending date in YYYY-MM-DD format')

    parser.add_argument('--cash', required=False, action='store',
                        type=float, default=50000,
                        help=('Cash to start with'))

    parser.add_argument('--smaperiod', required=False, action='store',
                        type=int, default=30,
                        help=('Period for the moving average'))

    parser.add_argument('--exitperiod', required=False, action='store',
                        type=int, default=5,
                        help=('Period for the exit control SMA'))

    parser.add_argument('--signal', required=False, action='store',
                        default=list(MAINSIGNALS.keys())[0], choices=MAINSIGNALS,
                        help=('Signal type to use for the main signal'))

    parser.add_argument('--exitsignal', required=False, action='store',
                        default=None, choices=EXITSIGNALS,
                        help=('Signal type to use for the exit signal'))

    # Plot options
    parser.add_argument('--plot', '-p', nargs='?', required=False,
                        metavar='kwargs', const=True,
                        help=('Plot the read data applying any kwargs passed\n'
                              '\n'
                              'For example:\n'
                              '\n'
                              '  --plot style="candle" (to plot candles)\n'))

    if pargs is not None:
        return parser.parse_args(pargs)

    return parser.parse_args()

In [9]:
def runstrat(args=None):
    args = parse_args(args)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)
    cerebro.broker.set_cash(args.cash)

    dkwargs = dict()
    if args.fromdate is not None:
        fromdate = datetime.datetime.strptime(args.fromdate, '%Y-%m-%d')
        dkwargs['fromdate'] = datetime.datetime(1995,1,3)

    if args.todate is not None:
        todate = datetime.datetime.strptime(args.todate, '%Y-%m-%d')
        dkwargs['todate'] = datetime.datetime(2014,12,31),

    # if dataset is None, args.data has been given
    datapath = '/home/lokendra/Downloads/orcl-1995-2014.txt'

    data = bt.feeds.BacktraderCSVData(dataname=args.data, **dkwargs)
    cerebro.adddata(data)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    cerebro.broker.setcommission(commission=0.001)
    cerebro.add_signal(MAINSIGNALS[args.signal],
                       SMACloseSignal, period=args.smaperiod)

    if args.exitsignal is not None:
        cerebro.add_signal(EXITSIGNALS[args.exitsignal],
                           SMAExitSignal,
                           p1=args.exitperiod,
                           p2=args.smaperiod)

    cerebro.run()
    printTradeAnalysis(strats[0].analyzers.ta.get_analysis())
    if args.plot:
        pkwargs = dict(style='bar')
        if args.plot is not True:  # evals to True but is not True
            npkwargs = eval('dict(' + args.plot + ')')  # args were passed
            pkwargs.update(npkwargs)

        cerebro.plot(**pkwargs)

In [10]:
if __name__ == '__main__':
    runstrat()

usage: ipykernel_launcher.py [-h] [--data DATA] [--fromdate FROMDATE]
                             [--todate TODATE] [--cash CASH]
                             [--smaperiod SMAPERIOD] [--exitperiod EXITPERIOD]
                             [--signal {longshort,longonly,shortonly}]
                             [--exitsignal {longexit,shortexit}]
                             [--plot [kwargs]]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/lokendra/.local/share/jupyter/runtime/kernel-9379a990-898c-4bd3-87c8-47ca3f06a156.json


SystemExit: 2